# Network Optimization AI Assistant
## Fine-tuning Llama-3-8B for Telecom Anomaly Detection

**Competition**: IndabaX Tunisia 2025 - Anomaly Solver Challenge  
**Model**: Llama-3-8B (4-bit quantized)  
**Technique**: LoRA Fine-tuning with Unsloth Optimization

In [1]:
# Environment Setup
!pip install -q pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install -q torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q unsloth evaluate datasets pandas

cryptography 44.0.3 is installed but cryptography<44 is required
Redoing requirement with just package name...
pyOpenSSL 25.0.0 is installed but pyOpenSSL<=24.2.1,>=19.1.0 is required
Redoing requirement with just package name...
rich 14.0.0 is installed but rich<14,>=12.4.4 is required
Redoing requirement with just package name...
numpy 1.26.4 is installed but numpy<3.0,>=2.0 is required
Redoing requirement with just package name...
fsspec 2025.3.2 is installed but fsspec[http]<=2025.3.0,>=2023.1.0 is required
Redoing requirement with just package name...
gymnasium 0.29.0 is installed but gymnasium>=1.0.0 is required
Redoing requirement with just package name...
scipy 1.15.2 is installed but scipy<1.14.0,>=1.7.0 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core[grpc]<3.0.0,>=2.16.0 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core<3.0.0dev,>=2.15.0 is require

In [2]:
# Configuration
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import pandas as pd
import numpy as np
import string  # <-- Added this import
from trl import SFTTrainer
from transformers import TrainingArguments

MODEL_NAME = "unsloth/llama-3-8b-bnb-4bit"
MAX_SEQ_LENGTH = 2048  # Using Unsloth's RoPE scaling
BATCH_SIZE = 2  # Fits Kaggle T4 GPU memory
GRAD_ACCUM_STEPS = 4
TRAIN_STEPS = 60  # Optimized for competition timeframe

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-30 21:27:58.799977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748640479.238487      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748640479.366913      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Data Preprocessing
def clean_text(text):
    """Basic text normalization"""
    return text.replace('\n', ' ').translate(str.maketrans('', '', string.punctuation))

def preprocess_function(examples):
    """Convert raw network KPIs to instruction format"""
    instructions = []
    inputs = []
    outputs = []
    
    for i in range(len(examples["ID"])):
        # Create input from all features
        input_parts = []
        for key in examples:
            if key not in ["ID", "Date", "network_labels", "improvement_solutions"]:
                value = examples[key][i]
                if isinstance(value, float) and np.isnan(value):
                    value = "N/A"
                input_parts.append(f"{key}: {value}")
        
        instructions.append("Analyze these network metrics and provide optimization solutions")
        inputs.append(clean_text(", ".join(input_parts)))
        
        # Handle test set missing solutions
        if "improvement_solutions" in examples:
            outputs.append(clean_text(str(examples["improvement_solutions"][i])))
        else:
            outputs.append("")
    
    return {"instruction": instructions, "input": inputs, "output": outputs}

In [4]:
# Dataset Preparation
train_dataset = load_dataset("csv", data_files="/kaggle/input/indabax-tunisia-2025-anomaly-solver-challenge-2/Train.csv", split="train")
test_dataset = load_dataset("csv", data_files="/kaggle/input/indabax-tunisia-2025-anomaly-solver-challenge-2/Test.csv", split="train")

# Preprocess
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [5]:
# Prompt Template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = "</s>"

def formatting_prompts_func(examples):
    """Format data into Alpaca-style prompts"""
    texts = []
    for instr, inp, outp in zip(examples["instruction"], examples["input"], examples["output"]):
        text = alpaca_prompt.format(instr, inp, outp) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [6]:
# Model Initialization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
)

# LoRA Configuration
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory optimization
)

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
# Training Setup
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM_STEPS,
        warmup_steps=5,
        max_steps=TRAIN_STEPS,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Start Training
trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/1000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.051400
2,3.025800
3,2.990500
4,2.914000
5,2.749200
6,2.506700
7,2.252800
8,1.981800
9,1.783100
10,1.608200


TrainOutput(global_step=60, training_loss=1.1444858153661093, metrics={'train_runtime': 2460.2601, 'train_samples_per_second': 0.39, 'train_steps_per_second': 0.024, 'total_flos': 3.887971157891482e+16, 'train_loss': 1.1444858153661093})

In [8]:
# Inference Function
def generate_predictions(dataset):
    """Generate solutions for test set"""
    model.eval()
    predictions = []
    for example in dataset:
        prompt = alpaca_prompt.format(
            example["instruction"],
            example["input"],
            ""
        )
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=200)
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred_text = pred_text.split("### Response:")[-1].strip()
        predictions.append(pred_text)
    return predictions

# Generate Predictions
test_predictions = generate_predictions(test_dataset)

In [9]:
# Create Submission
submission_df = pd.DataFrame({
    "ID": test_dataset["ID"],
    "improvement_solutions": test_predictions,
})

submission_df.to_csv("network_optimization_solutions.csv", index=False)
print("Submission file created successfully!")

Submission file created successfully!


In [10]:
# Model Export for Deployment
model.save_pretrained("network_optimizer")
tokenizer.save_pretrained("network_optimizer")

# Compress for download
!zip -r network_optimizer.zip network_optimizer/

  adding: network_optimizer/ (stored 0%)
  adding: network_optimizer/special_tokens_map.json (deflated 71%)
  adding: network_optimizer/adapter_config.json (deflated 56%)
  adding: network_optimizer/tokenizer.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 85%)
  adding: network_optimizer/README.md (deflated 66%)
  adding: network_optimizer/adapter_model.safetensors (deflated 7%)
  adding: network_optimizer/tokenizer_config.json (deflated 96%)
